## **설치**

In [ ]:
pip install selenium

In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install BeautifulSoup

# **인스타그램 크롤링 함수**

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time # 크롤링 중간중간 시간 지연
import re # 정규식 처리
import time 
import re

# 검색어에 대한 링크 생성
def insta_searching(word, option):
    if option == 'tag':
        url = "https://www.instagram.com/explore/tags/" + str(word)
    elif option == 'location':
        url = "https://www.instagram.com/explore/locations/" + str(word)
    return url

# 첫 번째 게시물 (인기 게시물 아니고 최근 게시물)
def select_first(driver): 
    first = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div[1]/div[1]/a/div[1]/div[2]')
    first.click()
    time.sleep(5)

# 한 게시물 크롤링 완료 후 다음 게시물 클릭
def move_next(driver):
    right = driver.find_element_by_css_selector('body > div._2dDPU.QPGbb.CkGkG > div.EfHg9 > div > div > div.l8mY4.feth3 > button')
    right.click()
    time.sleep(2)

In [3]:
import re
from bs4 import BeautifulSoup

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 본문 텍스트 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '

    # 본문 내 해시태그 가져오기
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 작성일자 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]

    # 좋아요 개수 가져오기
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1]
    except:
        like = 0
    
    # 위치
    try:
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    
    # 크롤링한 데이터 리스트 형태로 반환
    data = [content, date, like, place, tags]
    return data  

# **본격적인 크롤링 과정, 데이터 수집하기: 인스타그램 로그인 및 크롤링 작업**

In [ ]:
import pandas as pd

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.instagram.com')
time.sleep(3)

# 인스타그램 로그인
email = '인스타그램 아이디'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '인스타그램 비밀번호'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(3)

# 팝업창들 스킵 (필요시 삭제)
skip_button1 = driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF')
skip_button1.click()

time.sleep(3)

# 팝업창들 스킵 (필요시 삭제)
skip_button2 = driver.find_element_by_css_selector('button.aOOlW.HoLwm')
skip_button2.click()


# 검색어 입력 및 본격적 크롤링
word = input("검색어: ")
word = str(word)
url = insta_searching(word, "tag")

# 검색 결과 페이지 열기
driver.get(url)
time.sleep(7)

# 첫번째 게시물 클릭
select_first(driver)

# 데이터 크롤링
results = []

# 수집할 게시물의 수
target = 60
for i in range(target):
    if i in [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]:
        print(i)

    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    
print(results[:2])

results_df = pd.DataFrame(results)
results_df.columns = ['review', 'date', 'likes', 'place', 'hashtags']
results_df.to_csv('파일이름.csv')
